In [52]:
using IntervalArithmetic
using LaTeXStrings
using Plots

In [53]:
include("./method1/index.jl")
include("./intervalPCM/index.jl")
include("./twofoldInterval/index.jl")
include("./twofoldIntervalPCM/index.jl")

print()

In [54]:
function laTeXMatrix(
        pattern,
        size::Tuple{Integer, Integer})::String
    m, n = size

    mat = "\\begin{bmatrix}"
    for i = 1:m, j = 1:n
        mat *= pattern(i, j)
        if j == n
            if i != m
                mat *= " \\\\ "
            end
        else
            mat *= " & "
        end
    end
    mat *= "\\end{bmatrix}"

    return mat
end

laTeXMatrix (generic function with 1 method)

In [55]:
function method1(A₁::Matrix{Interval{T}}, A₂::Matrix{Interval{T}}) where {T <: Real}
    _, n = size(A₁)

    A = intervalPCM2TwofoldIntervalPCM(A₁, A₂)
    println("A = $(twofoldIntervalMatrixLaTeXString(A))"); println()

    result = solveLP_m1(A)

    resultₛₜᵣ = lPResultLaTeXString_m1(result)
    println("$(laTeXMatrix((i,j) -> "\\left[ w_$i^\\text{L}, w_$i^\\text{U} \\right]", (n, 1))) = $(resultₛₜᵣ.W)"); println()
    println("""
        $(laTeXMatrix((i,j) -> "w_$i^{\\text{L}-}", (n, 1))) = $(resultₛₜᵣ.wᴸ⁻), ~~
        $(laTeXMatrix((i,j) -> "w_$i^{\\text{U}-}", (n, 1))) = $(resultₛₜᵣ.wᵁ⁻), ~~
        $(laTeXMatrix((i,j) -> "w_$i^{\\text{L}+}", (n, 1))) = $(resultₛₜᵣ.wᴸ⁺), ~~
        $(laTeXMatrix((i,j) -> "w_$i^{\\text{U}+}", (n, 1))) = $(resultₛₜᵣ.wᵁ⁺)
        """); println()
    println("""
        $(laTeXMatrix((i,j) -> "\\varepsilon_$i^\\text{L}", (n, 1))) = $(resultₛₜᵣ.εᴸ), ~~
        $(laTeXMatrix((i,j) -> "\\varepsilon_$i^\\text{U}", (n, 1))) = $(resultₛₜᵣ.εᵁ)
        """); println()

    Â = updatePCM_m1(A, result)

    return Â
end

method1 (generic function with 1 method)

# example 1

In [56]:
A₁ = [
    1..1 (1/2)..2 (1/2)..2
    (1/2)..2 1..1 (1/2)..2
    (1/2)..2 (1/2)..2 1..1
]
_, n = size(A₁)
A₂ = fill(1..1, (n, n))
for i = 1:n, j = 1:n
    if i == j continue end
    A₂[i,j] = (A₁[i,j].lo * 4 / 5)..(A₁[i,j].hi * 5 / 4)
end

println("A_1 = A_2 = $(intervalMatrixLaTeXString(A₁))"); println()

Â = method1(A₁, A₂)

println("\\mathscr{A}^1 = $(twofoldIntervalMatrixLaTeXString(Â))")

A_1 = A_2 = \begin{bmatrix} \left[ 1.0, 1.0 \right] & \left[ 0.5, 2.0 \right] & \left[ 0.5, 2.0 \right] \\ \left[ 0.5, 2.0 \right] & \left[ 1.0, 1.0 \right] & \left[ 0.5, 2.0 \right] \\ \left[ 0.5, 2.0 \right] & \left[ 0.5, 2.0 \right] & \left[ 1.0, 1.0 \right] \end{bmatrix}

A = \begin{bmatrix} \left[   1.0, \left[   1.0,   1.0 \right],   1.0 \right] & \left[   0.4, \left[   0.5,   2.0 \right],   2.5 \right] & \left[   0.4, \left[   0.5,   2.0 \right],   2.5 \right] \\ \left[   0.4, \left[   0.5,   2.0 \right],   2.5 \right] & \left[   1.0, \left[   1.0,   1.0 \right],   1.0 \right] & \left[   0.4, \left[   0.5,   2.0 \right],   2.5 \right] \\ \left[   0.4, \left[   0.5,   2.0 \right],   2.5 \right] & \left[   0.4, \left[   0.5,   2.0 \right],   2.5 \right] & \left[   1.0, \left[   1.0,   1.0 \right],   1.0 \right] \end{bmatrix}

\begin{bmatrix}\left[ w_1^\text{L}, w_1^\text{U} \right] \\ \left[ w_2^\text{L}, w_2^\text{U} \right] \\ \left[ w_3^\text{L}, w_3^\text{U} \right]\end{bmatrix} = \begin{bmatrix}\left[ 0.19, 0.476 \right] \\ \left[ 0.19, 0.476 \right] \\ \left[ 0.19, 0.476 \right]\end{bmatrix}

\begin{bmatrix}w_1^{\text{L}-} \\ w_2^{\text{L}-} \\ w_3^{\text{L}-}\end{bmatrix} = \begin{bmatrix} 0.19  \\  0.19  \\  0.19 \end{bmatrix}, ~~
\begin{bmatrix}w_1^{\text{U}-} \\ w_2^{\text{U}-} \\ w_3^{\text{U}-}\end{bmatrix} = \begin{bmatrix} 0.381  \\  0.381  \\  0.381 \end{bmatrix}, ~~
\begin{bmatrix}w_1^{\text{L}+} \\ w_2^{\text{L}+} \\ w_3^{\text{L}+}\end{bmatrix} = \begin{bmatrix} 0.19  \\  0.19  \\  0.19 \end{bmatrix}, ~~
\begin{bmatrix}w_1^{\text{U}+} \\ w_2^{\text{U}+} \\ w_3^{\text{U}+}\end{bmatrix} = \begin{bmatrix} 0.381  \\  0.381  \\  0.381 \end{bmatrix}


\begin{bmatrix}\varepsilon_1^\text{L} \\ \varepsilon_2^\text{L} \\ \varepsilon_3^\text{L}\end{bmatrix} = \begin{bmatrix} 0.0  \\  0.0  \\  0.0 \en

# example 2

In [57]:
A₁ = [
    1..1 1..2 (2/3)..(4/3) 1..(4/3)
    (1/2)..1 1..1 (4/9)..1 (2/3)..1
    (3/4)..(3/2) 1..(9/4) 1..1 1..(3/2)
    (3/4)..1 1..(3/2) (2/3)..1 1..1
]
_, n = size(A₁)
A₂ = fill(1..1, (n, n))
for i = 1:n, j = 1:n
    if i == j continue end
    A₂[i,j] = (A₁[i,j].lo * 4 / 5)..(A₁[i,j].hi * 5 / 4)
end

Â = method1(A₁, A₂)

println("\\mathscr{A}^1 = $(twofoldIntervalMatrixLaTeXString(Â))")

A = \begin{bmatrix} \left[   1.0, \left[   1.0,   1.0 \right],   1.0 \right] & \left[   0.8, \left[   1.0,   2.0 \right],   2.5 \right] & \left[ 0.533, \left[ 0.667, 1.333 \right], 1.667 \right] & \left[   0.8, \left[   1.0, 1.333 \right], 1.667 \right] \\ \left[   0.4, \left[   0.5,   1.0 \right],  1.25 \right] & \left[   1.0, \left[   1.0,   1.0 \right],   1.0 \right] & \left[ 0.356, \left[ 0.444,   1.0 \right],  1.25 \right] & \left[ 0.533, \left[ 0.667,   1.0 \right],  1.25 \right] \\ \left[   0.6, \left[  0.75,   1.5 \right], 1.875 \right] & \left[   0.8, \left[   1.0,  2.25 \right], 2.812 \right] & \left[   1.0, \left[   1.0,   1.0 \right],   1.0 \right] & \left[   0.8, \left[   1.0,   1.5 \right], 1.875 \right] \\ \left[   0.6, \left[  0.75,   1.0 \right],  1.25 \right] & \left[   0.8, \left[   1.0,   1.5 \right], 1.875 \right] & \left[ 0.533, \left[ 0.667,   1.0 \right],  1.25 \right] & \left[   1.0, \left[   1.0,   1.0 \right],   1.0 \right] \end{bmatrix}

\begin{bmatrix}\left

# example 3

In [58]:
A = [
    1 2 4 4
    1/2 1 2 2
    1/4 1/2 1 1
    1/4 1/2 1 1
]

A₁ = randamizedIntervalPCM(A, 1234, 3e-2)
A₂ = randamizedIntervalPCM(A, 134, 3e-2)

Â = method1(A₁, A₂)

println("\\mathscr{A}^1 = $(twofoldIntervalMatrixLaTeXString(Â))")

A = \begin{bmatrix} \left[   1.0, \left[   1.0,   1.0 \right],   1.0 \right] & \left[ 1.981, \left[ 1.998,  2.01 \right], 2.033 \right] & \left[ 3.974, \left[ 3.974,  4.05 \right], 4.109 \right] & \left[ 3.958, \left[ 3.962, 4.041 \right], 4.048 \right] \\ \left[ 0.492, \left[ 0.497,   0.5 \right], 0.505 \right] & \left[   1.0, \left[   1.0,   1.0 \right],   1.0 \right] & \left[ 1.944, \left[ 1.945, 2.015 \right], 2.048 \right] & \left[ 1.942, \left[ 1.971, 2.033 \right], 2.045 \right] \\ \left[ 0.243, \left[ 0.247, 0.252 \right], 0.252 \right] & \left[ 0.488, \left[ 0.496, 0.514 \right], 0.515 \right] & \left[   1.0, \left[   1.0,   1.0 \right],   1.0 \right] & \left[ 0.972, \left[ 0.983, 1.021 \right], 1.022 \right] \\ \left[ 0.247, \left[ 0.247, 0.252 \right], 0.253 \right] & \left[ 0.489, \left[ 0.492, 0.507 \right], 0.515 \right] & \left[ 0.978, \left[ 0.979, 1.017 \right], 1.029 \right] & \left[   1.0, \left[   1.0,   1.0 \right],   1.0 \right] \end{bmatrix}

\begin{bmatrix}\left

# example 4

In [59]:
A = [
    1 2 3 10
    1/2 1 3 8
    1/3 1/3 1 3
    1/10 1/8 1/3 1
]

A₁ = randamizedIntervalPCM(A, 124, 4e-1)
A₂ = randamizedIntervalPCM(A, 411, 4e-1)

Â = method1(A₁, A₂)

println("\\mathscr{A}^1 = $(twofoldIntervalMatrixLaTeXString(Â))")

A = \begin{bmatrix} \left[   1.0, \left[   1.0,   1.0 \right],   1.0 \right] & \left[ 1.415, \left[ 1.547, 2.232 \right], 2.501 \right] & \left[ 2.233, \left[ 2.252, 3.723 \right],  4.04 \right] & \left[ 8.141, \left[ 8.746, 12.652 \right], 13.465 \right] \\ \left[   0.4, \left[ 0.448, 0.646 \right], 0.707 \right] & \left[   1.0, \left[   1.0,   1.0 \right],   1.0 \right] & \left[ 2.122, \left[  2.28, 3.056 \right], 3.644 \right] & \left[ 5.559, \left[ 6.481, 8.402 \right], 10.701 \right] \\ \left[ 0.248, \left[ 0.269, 0.444 \right], 0.448 \right] & \left[ 0.274, \left[ 0.327, 0.439 \right], 0.471 \right] & \left[   1.0, \left[   1.0,   1.0 \right],   1.0 \right] & \left[ 2.174, \left[ 2.759, 3.356 \right], 3.774 \right] \\ \left[ 0.074, \left[ 0.079, 0.114 \right], 0.123 \right] & \left[ 0.093, \left[ 0.119, 0.154 \right],  0.18 \right] & \left[ 0.265, \left[ 0.298, 0.362 \right],  0.46 \right] & \left[   1.0, \left[   1.0,   1.0 \right],   1.0 \right] \end{bmatrix}

\begin{bmatrix}\l